In [24]:
import urllib
from bs4 import BeautifulSoup
import re
import xlwt

savePath ="安徽新闻网链接.xls"
def main():
    url = "http://www.ahnews.com.cn/"
    collectUrl(url)
    
#获得安徽新闻网的主要版块和其主网址
def collectUrl(url):
    header = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
    }
    req = urllib.request.Request(url=url,headers=header)
    res = urllib.request.urlopen(req)
    html = res.read().decode("utf-8")
    pageContent = BeautifulSoup(html,"html.parser")
    pageContent = str(pageContent)
    news =re.compile(r'<li> <a href="(.*?)/pc/lay/(.*?).html" target="_blank">(.*?)</a></li>')
    data = re.findall(news,pageContent)
    #存储相关板块的链接
    book = xlwt.Workbook(encoding = "utf-8",style_compression=0)
    sheet = book.add_sheet("ahNews",cell_overwrite_ok=True)
    #单元格插入的起始位置
    mainAera = ("版块","网址")
    for i in range(len(mainAera)):
        sheet.write(0,i,mainAera[i])
    for datacell,i in zip(data,range(len(data))):
        mainUrl = datacell[0]+'/pc/lay/'+datacell[1]+'.html'
        print(i,datacell[2],':',mainUrl)
        sheet.write(i+1,0,datacell[2])
        sheet.write(i+1,1,mainUrl)
        book.save("anhuiNew.xls")
main()
    

0 政务 : http://www.ahnews.com.cn/zhengwu/pc/lay/node_558.html
1 评论 : http://www.ahnews.com.cn/wangping/pc/lay/node_578.html
2 理论 : http://www.ahnews.com.cn/pinglun/pc/lay/node_564.html
3 党建 : http://www.ahnews.com.cn/dangjian/pc/lay/node_548.html
4 问政 : http://www.ahnews.com.cn/wenzheng/pc/lay/node_559.html
5 文化 : http://www.ahnews.com.cn/wh/pc/lay/node_500.html
6 教育 : http://www.ahnews.com.cn/jiaoyu/pc/lay/node_552.html
7 经济 : http://www.ahnews.com.cn/jingji/pc/lay/node_560.html
8 房产 : http://www.ahnews.com.cn/fangchan/pc/lay/node_544.html
9 徽商 : http://www.ahnews.com.cn/huishang/pc/lay/node_561.html
10 旅游 : http://www.ahnews.com.cn/lvyou/pc/lay/node_546.html
11 专题 : http://www.ahnews.com.cn/zt/pc/lay/node_497.html
12 合肥 : http://www.ahnews.com.cn/hefei/pc/lay/node_565.html
13 安徽 : http://www.ahnews.com.cn/anhui/pc/lay/node_562.html
14 国内 : http://www.ahnews.com.cn/guonei/pc/lay/node_543.html
15 国际 : http://www.ahnews.com.cn/gj/pc/lay/node_503.html
16 滚动 : http://www.ahnews.com.cn/gund

In [4]:
#reda url-content
import xlrd
import urllib
import re
import jieba
import time 
from rich.progress import track

def SaveNews(path,str):
    #path指的是文件存储的文件位置及文件名
    #str是需要存储的内容，追加的方式存储
    with open(path,"a+") as f:
        f.write(str)
def clearData(Data):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    Data = re.sub(pattern,"",Data)
    Data = re.sub(r'新华社',"",Data)
    return Data
def collectContent(NewPath):
    header = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
    }
    workbook = xlrd.open_workbook(NewPath)
    urlsheet = workbook.sheet_by_index(0)
    print("共%d篇新闻稿"%(urlsheet.nrows-1))
    for rx in track(range(1,urlsheet.nrows)):
        newsUrl = urlsheet.row(rx)[0].value
        req = urllib.request.Request(url=newsUrl,headers=header)
        res = urllib.request.urlopen(req)
        html = res.read().decode("utf-8")
        pageContent = BeautifulSoup(html,"html.parser")
        pageContent = str(pageContent)
        finalContent = clearData(pageContent)
        time.sleep(1)
        SaveNews("AnHuiNews.txt",finalContent)
    collectContent("安徽新闻网链接.xls")


共65篇新闻稿


Output()

下载完成,可进行下一步分析


In [63]:
#对每个分网址进行处理
import urllib
from bs4 import BeautifulSoup
def dealSingleNet(url):
    header = {
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36 Edg/87.0.664.75"
    }
    req = urllib.request.Request(url=url,headers=header)
    res = urllib.request.urlopen(req)
    html = res.read().decode("utf-8")
    pageContent = BeautifulSoup(html,"html.parser")
    pageContent = str(pageContent)
    pageRule = re.compile(r'<ul class="pages" current="1" pages="(.*?)">')
    totalpage = re.findall(pageRule,pageContent)
    print(totalpage)
    workbook = xlrd.open_workbook("anhuiNew.xls")
    urlsheet = workbook.sheet_by_index(0)
    print("开始组装需要搜索的网页")
    focus=["要闻","评论","政务","党建","文化","经济","徽商","专题","安徽","国际","视觉","理论","问政","教育","房产","旅游","合肥","国内","滚动"]
    #存储相关板块的链接
    bolckUrl = xlwt.Workbook(encoding = "utf-8",style_compression=0)
    sheet = bolckUrl.add_sheet("bolckUrl",cell_overwrite_ok=True)
    #单元格插入的起始位置
    mainAera = ("版块","网址")
    for i in range(len(mainAera)):
        sheet.write(0,i,mainAera[i])
    for rx in range(1,urlsheet.nrows):
        bolck = urlsheet.row(rx)[0].value
        sheet.write(0,rx-1,bolck)
        url = urlsheet.row(rx)[1].value
        req = urllib.request.Request(url=url,headers=header)
        res = urllib.request.urlopen(req)
        html = res.read().decode("utf-8")
        pageContent = BeautifulSoup(html,"html.parser")
        pageContent = str(pageContent)
        pageRule = re.compile(r'<ul class="pages" current="1" pages="(.*?)">')
        totalpage = re.findall(pageRule,pageContent)
        if totalpage:
            print("共%s页新闻稿"%(totalpage))
            sheet.write(1,rx-1,url)
            for i in range(2,int(totalpage[0])+1):
                temUrl = url.replace(".html","_"+str(i)+".html")
                sheet.write(i,rx-1,temUrl)
                print(temUrl)
        else:
            print(bolck)
    bolckUrl.save("bolckUrl.xls")
dealSingleNet('http://www.ahnews.com.cn/wangping/pc/lay/node_578.html')

/pc/lay/node_562_42.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_43.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_44.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_45.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_46.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_47.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_48.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_49.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_50.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_51.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_52.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_53.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_54.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_55.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_56.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_57.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_58.html
http://www.ahnews.com.cn/anhui/pc/lay/node_562_59.html
http://www.ahnews.com.cn/anhui/pc/lay/no

In [53]:
a = "24"
int(a)

24

In [56]:
a = []
a == True

False